In [1]:
# Dependencies
import tweepy
import json
import pandas as pd
import datetime as dt
from config import consumer_key, consumer_secret, access_token, access_token_secret

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [2]:
# Target User
target_user = "elonmusk"

# Tweet Texts
tweet_texts = []

# Create a loop to iteratively run API requests
for x in range(1, 200): #20 per page

    # Get all tweets from home feed (for each page specified)
    public_tweets = api.user_timeline(target_user, page=x)

    # Loop through all tweets
    for tweet in public_tweets:

        # Print Tweet
        #print(tweet["text"],tweet["created_at"])

        # Store Tweet in Array
        tweet_texts.append([tweet["created_at"],tweet["text"],tweet["favorite_count"],tweet["retweet_count"]])
        #tweet_texts.append(tweet)

In [3]:
# Print the Tweet Count
print(f"Tweet Count: {len(tweet_texts)}")

Tweet Count: 3226


In [4]:
tweetData_df = pd.DataFrame(tweet_texts)
tweetData_df.columns = ["Tweet_Date", "Tweet_Text", "Favorite_Count", "Retweet_Count"]
tweetData_df.head()

,Tweet_Date,Tweet_Text,Favorite_Count,Retweet_Count
0,Mon Oct 29 22:06:31 +0000 2018,Legally required officers of a corporation are...,33088,1233
1,Mon Oct 29 21:44:07 +0000 2018,"@FredericLambert Good point, might put that on...",2293,40
2,Mon Oct 29 20:43:53 +0000 2018,Deleted my Tesla titles last week to see what ...,128422,9747
3,Sat Oct 27 03:54:33 +0000 2018,@SnazzyQ @FredericLambert Signing off Twitter ...,2568,117
4,Sat Oct 27 03:50:03 +0000 2018,"@SnazzyQ @FredericLambert Don’t worry, we’ll m...",1523,39


In [5]:
tweetData_df.to_csv("MuskTweetData.csv", encoding="utf-8")

In [6]:
tweet_ETL = "MuskTweetData.csv"
tweet_ETL_df = pd.read_csv(tweet_ETL).drop(['Unnamed: 0'],axis=1)
tweet_ETL_df.head()

,Tweet_Date,Tweet_Text,Favorite_Count,Retweet_Count
0,Mon Oct 29 22:06:31 +0000 2018,Legally required officers of a corporation are...,33088,1233
1,Mon Oct 29 21:44:07 +0000 2018,"@FredericLambert Good point, might put that on...",2293,40
2,Mon Oct 29 20:43:53 +0000 2018,Deleted my Tesla titles last week to see what ...,128422,9747
3,Sat Oct 27 03:54:33 +0000 2018,@SnazzyQ @FredericLambert Signing off Twitter ...,2568,117
4,Sat Oct 27 03:50:03 +0000 2018,"@SnazzyQ @FredericLambert Don’t worry, we’ll m...",1523,39


In [7]:
tweet_ETL_df["Popular_Sum"] = tweet_ETL_df["Favorite_Count"] + tweet_ETL_df["Retweet_Count"]
tweet_ETL_df.head()

,Tweet_Date,Tweet_Text,Favorite_Count,Retweet_Count,Popular_Sum
0,Mon Oct 29 22:06:31 +0000 2018,Legally required officers of a corporation are...,33088,1233,34321
1,Mon Oct 29 21:44:07 +0000 2018,"@FredericLambert Good point, might put that on...",2293,40,2333
2,Mon Oct 29 20:43:53 +0000 2018,Deleted my Tesla titles last week to see what ...,128422,9747,138169
3,Sat Oct 27 03:54:33 +0000 2018,@SnazzyQ @FredericLambert Signing off Twitter ...,2568,117,2685
4,Sat Oct 27 03:50:03 +0000 2018,"@SnazzyQ @FredericLambert Don’t worry, we’ll m...",1523,39,1562


In [8]:
#tweetData_df_date = tweetData_df.Tweet_Date=pd.to_datetime(tweetData_df.Tweet_Date).dt.strftime('%Y-%M-%d %H:%M:%S')
tweet_ETL_dt_df = tweet_ETL_df.copy()
tweet_ETL_dt_df['Date'] = tweet_ETL_df.Tweet_Date=pd.to_datetime(tweet_ETL_df.Tweet_Date).dt.strftime('%Y-%m-%d')
tweet_ETL_dt_df.head()

,Tweet_Date,Tweet_Text,Favorite_Count,Retweet_Count,Popular_Sum,Date
0,Mon Oct 29 22:06:31 +0000 2018,Legally required officers of a corporation are...,33088,1233,34321,2018-10-29
1,Mon Oct 29 21:44:07 +0000 2018,"@FredericLambert Good point, might put that on...",2293,40,2333,2018-10-29
2,Mon Oct 29 20:43:53 +0000 2018,Deleted my Tesla titles last week to see what ...,128422,9747,138169,2018-10-29
3,Sat Oct 27 03:54:33 +0000 2018,@SnazzyQ @FredericLambert Signing off Twitter ...,2568,117,2685,2018-10-27
4,Sat Oct 27 03:50:03 +0000 2018,"@SnazzyQ @FredericLambert Don’t worry, we’ll m...",1523,39,1562,2018-10-27


In [9]:
popular = tweet_ETL_dt_df.groupby(['Date','Tweet_Text'], as_index=False).agg({'Popular_Sum':'max'})#.rename(columns={'ride_id':'rides', 'fare':'avg_fare'})
popular.head()

,Date,Tweet_Text,Popular_Sum
0,2017-02-05,@CycleTrades @rtoro20 People should push to ha...,1971
1,2017-02-05,"@FredericLambert Yeah, it is hard to convey co...",635
2,2017-02-05,"@FredericLambert Yes, but Model 3 is the overw...",629
3,2017-02-05,@MacTechGenius App update is about two weeks o...,457
4,2017-02-05,@camdtor @trylks Almost ready,599


In [10]:
popular.to_csv("MuskTweetPopular.csv", encoding="utf-8")